In [1]:
pip install streamlit langchain langchain-community langchain-huggingface faiss-cpu pandas sentence-transformers

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-huggingface to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 68.9 MB/s eta 0:00:0000:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 78.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 73.3 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 450.8/450.8 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 81.7 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 81.5 MB/s eta 0:00:00:00:010:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 78.7 MB/s eta 0:00:00:00:

In [3]:
# --- STEP 1: Install & Start Ollama (Run this once) ---
import subprocess
import time

print("⬇️ Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

print("\n🚀 Starting Ollama Server...")
# Run Ollama in the background
process = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

# Give it 10 seconds to wake up
time.sleep(10)

print("\n🧠 Downloading Llama 3 Model (This takes a few minutes)...")
!ollama pull llama3

print("\n✅ Setup Complete! Now you can run your RAG code.")

⬇️ Installing Ollama...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%                                               36.1%########################                                     52.6%######################################################      95.2%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

🚀 Starting Ollama Server...

🧠 Downloading Llama 3 Model (This takes a few minutes)...
pulling manifest ⠋ 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 6a0746a1ec1a:   0% ▕                  ▏ 1.5 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   2% ▕                  ▏  75 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   3% ▕                  ▏ 147 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   4% ▕                  ▏ 183 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   5% ▕                  ▏ 255 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   7% ▕█                 ▏ 334 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:   8% ▕█                 ▏ 375 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:  10% ▕█                 ▏ 455 MB/4.7 GB                  pulling manifest 
pulling 6a0746a1ec1a:  12% ▕██                ▏ 536 MB/4.7 GB            

In [8]:
import os
import json
import pandas as pd
from langchain_community.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
# We use ChatOllama for local privacy (Essential for clinical data assignments)
from langchain_community.chat_models import ChatOllama 

class ClinicalRAG:
    def __init__(self):
        self.vector_store = None
        self.retriever = None
        self.chain = None
        
        # 1. Embeddings: Converts text to numbers for search
        print("Initializing Embeddings Model...")
        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        
        # 2. LLM: The brain of the operation
        print("Connecting to Local LLM (Ollama)...")
        self.llm = ChatOllama(model="llama3", temperature=0.2) 
        
    def load_json_content(self, file_path):
        """
        Reads a single JSON file and combines 'input1'...'input6' into a single note.
        """
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # --- CUSTOM LOGIC FOR YOUR DATASET ---
            # We combine the separate fields into one readable clinical text.
            parts = []
            
            if 'input1' in data: 
                parts.append(f"CHIEF COMPLAINT: {data['input1']}")
            
            if 'input2' in data: 
                parts.append(f"HISTORY OF PRESENT ILLNESS: {data['input2']}")
            
            if 'input3' in data: 
                parts.append(f"PAST MEDICAL HISTORY: {data['input3']}")
            
            if 'input4' in data: 
                parts.append(f"FAMILY HISTORY: {data['input4']}")

            if 'input5' in data: 
                parts.append(f"PHYSICAL EXAM: {data['input5']}")
                
            if 'input6' in data: 
                parts.append(f"LABS AND VITALS: {data['input6']}")

            # If we found your specific keys, join them.
            if parts:
                content = "\n\n".join(parts)
            else:
                # Fallback: Dump everything if 'input1' etc are missing
                content = json.dumps(data)
                
            return content
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return None

    def ingest_data(self, root_path):
        """
        Recursively crawls the folder structure.
        Crucial Feature: Captures the FOLDER NAME as the Disease Category.
        """
        print(f"Scanning directory tree: {root_path}")
        
        docs_list = []
        
        # os.walk is the crawler. It goes into every sub-folder.
        for root, dirs, files in os.walk(root_path):
            # Calculate relative path to create a clean category name
            relative_path = os.path.relpath(root, root_path)
            
            if relative_path == ".":
                category_label = "General"
            else:
                category_label = relative_path.replace(os.sep, " > ")

            # Filter for JSON files
            json_files = [f for f in files if f.endswith('.json')]
            
            if json_files:
                print(f"📂 Found {len(json_files)} docs in: {category_label}")
                
                # LIMIT FOR ASSIGNMENT SPEED:
                # We only take the first 5 files per folder to ensure the vector store builds quickly.
                for file in json_files[:5]:
                    full_path = os.path.join(root, file)
                    text_content = self.load_json_content(full_path)
                    
                    if text_content:
                        docs_list.append({
                            # We inject the category context directly into the text
                            "text": f"Condition Category: {category_label}.\n\n{text_content}",
                            "source": file,
                            "category": category_label
                        })

        if not docs_list:
            return "No JSON files found! Check the path."

        # Convert to DataFrame for LangChain processing
        df = pd.DataFrame(docs_list)
        print(f"Total documents processed: {len(df)}")

        # Load into LangChain
        loader = DataFrameLoader(df, page_content_column="text")
        documents = loader.load()

        # Split Text: Clinical notes are long, we need chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        texts = text_splitter.split_documents(documents)

        # Create Vector Store (FAISS)
        print("Creating vector index... this may take a moment.")
        self.vector_store = FAISS.from_documents(texts, self.embeddings)
        self.retriever = self.vector_store.as_retriever(search_kwargs={"k": 4}) 
        
        self.setup_chain()
        return "Success"

    def setup_chain(self):
        """
        Sets up the RAG pipeline: Retrieval -> Prompt -> LLM
        """
        template = """
        You are a clinical assistant AI designed for Diagnostic Reasoning (DiReCT).
        Use the following retrieved clinical notes to answer the question.
        
        Instructions:
        1. Base your answer ONLY on the provided context.
        2. Pay attention to the 'Condition Category' (e.g., Asthma > Allergic).
        3. Be professional and concise.
        
        Context:
        {context}

        Question:
        {question}

        Answer:
        """
        
        prompt = ChatPromptTemplate.from_template(template)
        output_parser = StrOutputParser()

        setup_and_retrieval = RunnableParallel(
            {"context": self.retriever, "question": RunnablePassthrough()}
        )

        self.chain = setup_and_retrieval | prompt | self.llm | output_parser

    def query(self, user_query):
        if not self.chain:
            return "Please ingest data first.", []
        
        # Return both the answer AND the source documents (for the UI)
        return self.chain.invoke(user_query), self.retriever.invoke(user_query)

# --- MAIN EXECUTION BLOCK (FOR TESTING) ---
if __name__ == "__main__":
    print("--- RAG Engine Test Mode (Fixed Inputs) ---")
    rag = ClinicalRAG()
    # You can hardcode your path here for testing
    path = input("Enter dataset path to test: ")
    if os.path.exists(path):
        rag.ingest_data(path)
        print("Ready. Type 'exit' to quit.")
        while True:
            q = input("Query: ")
            if q == 'exit': break
            ans, _ = rag.query(q)
            print("Answer:", ans)
    else:
        print("Path invalid.")

--- RAG Engine Test Mode (Fixed Inputs) ---
Initializing Embeddings Model...
Connecting to Local LLM (Ollama)...


Enter dataset path to test:  /kaggle/input/sample/Finished


Scanning directory tree: /kaggle/input/sample/Finished
📂 Found 16 docs in: Pneumonia > Bacterial Pneumonia
📂 Found 4 docs in: Pneumonia > Viral Pneumonia
📂 Found 15 docs in: Stroke > Hemorrhagic Stroke
📂 Found 13 docs in: Stroke > Ischemic Stroke
📂 Found 20 docs in: Peptic Ulcer Disease > Gastric Ulcers
📂 Found 8 docs in: Peptic Ulcer Disease > Duodenal Ulcers
📂 Found 3 docs in: Asthma > Non-Allergic Asthma
📂 Found 2 docs in: Asthma > Asthma
📂 Found 2 docs in: Asthma > COPD Asthma
📂 Found 4 docs in: Asthma > Severe Asthma Exacerbation
📂 Found 1 docs in: Asthma > Allergic Asthma
📂 Found 1 docs in: Asthma > Cough-Variant Asthma
📂 Found 1 docs in: Pituitary Disease > Pituitary Microadenoma
📂 Found 11 docs in: Pituitary Disease > Pituitary Macroadenoma
📂 Found 3 docs in: Migraine > Migraine With Aura
📂 Found 1 docs in: Migraine > Migraine Without Aura
📂 Found 6 docs in: Diabetes > Type II Diabetes
📂 Found 7 docs in: Diabetes > Type I Diabetes
📂 Found 22 docs in: Acute Coronary Syndrome > U

Query:  What are the common symptoms listed for Heart Failure?


Answer: Based on the provided clinical notes, the common symptoms listed for Heart Failure are:

* Shortness of breath
* Lower extremity edema
* Chest pressure
* Abdominal pain
* Weight gain
* General fatigue
* Fatigue

These symptoms are mentioned across multiple patient records with a condition category of "Heart Failure".


Query:  exit


In [13]:
import subprocess
import time

# 1. Install Ollama
print("⬇️ Installing Ollama...")
!curl -fsSL https://ollama.com/install.sh | sh

# 2. Start Server in Background
print("\n🚀 Starting Ollama Server...")
process = subprocess.Popen("ollama serve", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)
time.sleep(5)  # Give it time to start

# 3. Pull Model
print("\n🧠 Downloading Llama 3 Model (This takes a few minutes)...")
!ollama pull llama3
print("\n✅ Ollama Ready!")

⬇️ Installing Ollama...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


>>> Cleaning up old version at /usr/local/lib/ollama
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%#                                                                 13.5%###########                                                  33.7%##########################                      72.8%
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.

🚀 Starting Ollama Server...

🧠 Downloading Llama 3 Model (This takes a few minutes)...
pulling manifest ⠋ 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


pulling manifest ⠙ pulling manifest ⠹ pulling manifest 
pulling 6a0746a1ec1a: 100% ▕██████████████████▏ 4.7 GB                         
pulling 4fa551d4f938: 100% ▕██████████████████▏  12 KB                         
pulling 8ab4849b038c: 100% ▕██████████████████▏  254 B                         
pulling 577073ffcc6c: 100% ▕██████████████████▏  110 B                         
pulling 3f8eb4da87fa: 100% ▕██████████████████▏  485 B                         
verifying sha256 digest 
writing manifest 
success 

✅ Ollama Ready!


In [14]:
%%writefile app.py
import streamlit as st
import os
import json
import pandas as pd

# Imports
from langchain_community.document_loaders import DataFrameLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser
from langchain_community.chat_models import ChatOllama 

# --- CLINICAL RAG CLASS ---
class ClinicalRAG:
    def __init__(self):
        self.vector_store = None
        self.retriever = None
        self.chain = None
        
        print("Initializing Embeddings Model...")
        self.embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
        
        print("Connecting to Local LLM (Ollama)...")
        self.llm = ChatOllama(model="llama3", temperature=0.2) 
        
    def load_json_content(self, file_path):
        """Reads a single JSON file and combines 'input1'...'input6'."""
        try:
            with open(file_path, 'r', encoding='utf-8') as f:
                data = json.load(f)
            
            # Combine distinct inputs into one note
            parts = []
            if 'input1' in data: parts.append(f"CHIEF COMPLAINT: {data['input1']}")
            if 'input2' in data: parts.append(f"HISTORY OF PRESENT ILLNESS: {data['input2']}")
            if 'input3' in data: parts.append(f"PAST MEDICAL HISTORY: {data['input3']}")
            if 'input4' in data: parts.append(f"FAMILY HISTORY: {data['input4']}")
            if 'input5' in data: parts.append(f"PHYSICAL EXAM: {data['input5']}")
            if 'input6' in data: parts.append(f"LABS AND VITALS: {data['input6']}")

            if parts:
                content = "\n\n".join(parts)
            else:
                content = json.dumps(data) # Fallback
                
            return content
        except Exception as e:
            print(f"Error reading {file_path}: {e}")
            return None

    def ingest_data(self, root_path):
        """Recursively crawls the folder structure."""
        print(f"Scanning directory tree: {root_path}")
        docs_list = []
        
        for root, dirs, files in os.walk(root_path):
            relative_path = os.path.relpath(root, root_path)
            
            if relative_path == ".":
                category_label = "General"
            else:
                category_label = relative_path.replace(os.sep, " > ")

            json_files = [f for f in files if f.endswith('.json')]
            
            if json_files:
                # Limit to 5 files per folder for speed
                for file in json_files[:5]:
                    full_path = os.path.join(root, file)
                    text_content = self.load_json_content(full_path)
                    
                    if text_content:
                        docs_list.append({
                            "text": f"Condition Category: {category_label}.\n\n{text_content}",
                            "source": file,
                            "category": category_label
                        })

        if not docs_list:
            return "No JSON files found!"

        df = pd.DataFrame(docs_list)
        loader = DataFrameLoader(df, page_content_column="text")
        documents = loader.load()

        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
        texts = text_splitter.split_documents(documents)

        print("Creating vector index...")
        self.vector_store = FAISS.from_documents(texts, self.embeddings)
        self.retriever = self.vector_store.as_retriever(search_kwargs={"k": 4}) 
        
        self.setup_chain()
        return "Success"

    def setup_chain(self):
        template = """
        You are a clinical assistant AI designed for Diagnostic Reasoning (DiReCT).
        Use the provided clinical context to answer the question.
        
        Context:
        {context}

        Question:
        {question}

        Answer:
        """
        prompt = ChatPromptTemplate.from_template(template)
        output_parser = StrOutputParser()
        setup_and_retrieval = RunnableParallel(
            {"context": self.retriever, "question": RunnablePassthrough()}
        )
        self.chain = setup_and_retrieval | prompt | self.llm | output_parser

    def query(self, user_query):
        if not self.chain:
            return "Please ingest data first.", []
        return self.chain.invoke(user_query), self.retriever.invoke(user_query)

# --- STREAMLIT UI ---
st.set_page_config(page_title="DiReCT: Clinical RAG", page_icon="🏥", layout="wide")
st.title("🏥 DiReCT: Diagnostic Reasoning")

if "rag_system" not in st.session_state:
    st.session_state.rag_system = ClinicalRAG()
if "messages" not in st.session_state:
    st.session_state.messages = []

with st.sidebar:
    st.header("📂 Data Source")
    # Default Kaggle Input Path
    default_path = "/kaggle/input/sample/Finished"
    data_path = st.text_input("Folder Path:", value=default_path)
    
    if st.button("Load & Index Data"):
        if data_path and os.path.exists(data_path):
            with st.spinner(f"Crawling {data_path}..."):
                result = st.session_state.rag_system.ingest_data(data_path)
                if result == "Success":
                    st.success("✅ Indexing Complete!")
                else:
                    st.error(result)
        else:
            st.error("⚠️ Path does not exist.")

for message in st.session_state.messages:
    with st.chat_message(message["role"]):
        st.markdown(message["content"])

if prompt := st.chat_input("Ask a clinical question..."):
    st.session_state.messages.append({"role": "user", "content": prompt})
    with st.chat_message("user"):
        st.markdown(prompt)

    with st.chat_message("assistant"):
        message_placeholder = st.empty()
        
        if st.session_state.rag_system.chain is None:
            message_placeholder.error("⚠️ Load data first.")
        else:
            with st.spinner("Analyzing..."):
                try:
                    response, sources = st.session_state.rag_system.query(prompt)
                    full_display = response + "\n\n---\n**📚 Evidence:**\n"
                    seen = set()
                    for doc in sources:
                        cat = doc.metadata.get('category', 'Unknown')
                        src = doc.metadata.get('source', 'Unknown')
                        if f"{cat}-{src}" not in seen:
                            full_display += f"- **{cat}**: `{src}`\n"
                            seen.add(f"{cat}-{src}")
                    
                    message_placeholder.markdown(full_display)
                    st.session_state.messages.append({"role": "assistant", "content": full_display})
                except Exception as e:
                    st.error(f"Error: {e}")

Overwriting app.py


In [ ]:
import subprocess
import urllib.request
import time

# 1. Get the Password
print("Password/Endpoint IP for LocalTunnel is:", urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

# 2. Start Streamlit
print("🚀 Starting Streamlit...")
streamlit_process = subprocess.Popen(["streamlit", "run", "app.py"])
time.sleep(5)

# 3. Expose to Internet (Added '-y' to auto-accept installation)
print("🔗 Creating Tunnel...")
!npx -y localtunnel --port 8501

Password/Endpoint IP for LocalTunnel is: 146.148.82.74
🚀 Starting Streamlit...



  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://172.19.2.2:8501
  External URL: http://146.148.82.74:8501

🔗 Creating Tunnel...


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙your url is: https://nasty-llamas-report.loca.lt


2025-11-30 18:06:10.802236: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764525970.827649     451 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764525970.835353     451 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'
/usr/local/lib/python3.11/dist-packages/pydantic/_internal/_generate_schema.p